<a href="https://colab.research.google.com/github/MushroomFleet/StableCascade-text2image/blob/main/240217_Pro_gated_colab_of_Stable_Cascade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pro Gated - Stable Cascade Txt2img
Original Author [@mkshing](https://github.com/mkshing) - Forked by Drift Johnson
- aims to run A100 to explore the potential of cascade with diffusers

In [1]:
# Check the GPU
!nvidia-smi

Sat Feb 17 02:20:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0              44W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

to save all images to google drive at the end of your session (optional)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# INSTALL Cascade

In [3]:
%cd /content/
!git lfs install
!git clone https://huggingface.co/spaces/multimodalart/stable-cascade
%cd /content/stable-cascade
!pip install -r /content/stable-cascade/requirements.txt
!pip3 uninstall torch torchvision torchaudio -y
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip uninstall torchdata torchtext -y
!pip install torchdata torchtext

/content
Git LFS initialized.
Cloning into 'stable-cascade'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (207/207), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 211 (delta 122), reused 0 (delta 0), pack-reused 4
Receiving objects: 100% (211/211), 78.76 KiB | 26.25 MiB/s, done.
Resolving deltas: 100% (122/122), done.
/content/stable-cascade
  Cloning https://github.com/kashif/diffusers.git (to revision diffusers-yield-callback) to /tmp/pip-req-build-qys_08eg
  Running command git clone --filter=blob:none --quiet https://github.com/kashif/diffusers.git /tmp/pip-req-build-qys_08eg
  Running command git checkout -b diffusers-yield-callback --track origin/diffusers-yield-callback
  Switched to a new branch 'diffusers-yield-callback'
  Branch 'diffusers-yield-callback' set up to track remote branch 'diffusers-yield-callback' from 'origin'.
  Resolved https://github.com/kashif/diffusers.git to commit c4d20110774dbcc6ada7e62b290500f8ad777f

# LOAD Cascade

In [4]:
import torch
import numpy as np
from diffusers import StableCascadeDecoderPipeline, StableCascadePriorPipeline
from diffusers.utils import numpy_to_pil

# Assuming the environment setup and model loading is done as per your script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

# Initialize pipelines (ensure this matches your environment's setup)
prior_pipeline = StableCascadePriorPipeline.from_pretrained("stabilityai/stable-cascade-prior", torch_dtype=dtype).to(device)
decoder_pipeline = StableCascadeDecoderPipeline.from_pretrained("stabilityai/stable-cascade", torch_dtype=dtype).to(device)


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/p

model_index.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

...

tokenizer/vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

scheduler/scheduler_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.39G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/36.8M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

# Notes  
- (4096x4096  requires 30.6gb) *A100 2 mins 30 secs
- (4096x2048  requires 25.9gb) *A100 1 min
- (2048x2048  requires 19.9gb) *A100 30 secs
- ~
- (2048x1536) *A100 in 20 seconds
- (2048x1152  requires 17.5gb) *A100 in 20 seconds

# Generation parameters

In [65]:
# @markdown ### **Positive Prompt**
pos_prompt = "2RAW photo of a musclebound chimpanzee wearing hiphop clothing and (black sunglasses), standing inside an impossible 3D labyrinth" # @param {type:"string"}
# @markdown ### **Negative Prompt**
neg_prompt = "drawing, sketch, painting, fake, monochrome" # @param {type:"string"}

# @markdown ### **Width**
Bwidth = 1024 # @param {type:"number"}
# @markdown ### **Height**
Bheight = 1024 # @param {type:"number"}

# @markdown ### **Width**
Cwidth = 2048 # @param {type:"number"}
# @markdown ### **Height**
Cheight = 1152 # @param {type:"number"}

# @markdown ### **Prior Guidance Scale**
Uprior_guidance_scale = 7.5 # @param {type:"number"}
# @markdown ### **Decoder Guidance Scale**
Udecoder_guidance_scale = 0.0 # @param {type:"number"}
# @markdown ### **Steps**
Unum_inference_steps = 50 # @param {type:"number"}

# batch over 1 caused tensor size mismatch
Unum_images_per_prompt = 1


# click to run & generate image

In [66]:
import random
import re


Random_seed = random.randint(0, 6969420)

def generate_image(prompt, negative_prompt, seed=Random_seed, width=Bwidth, height=Bheight, prior_guidance_scale=Uprior_guidance_scale, decoder_guidance_scale=Udecoder_guidance_scale, num_inference_steps=Unum_inference_steps, num_images_per_prompt=Unum_images_per_prompt):
    # Set seed for reproducibility
    Random_seed = random.randint(0, 6969420)
    generator = torch.Generator(device=device).manual_seed(seed)

    # Generate with Prior
    prior_results = prior_pipeline(
        prompt=prompt,
        negative_prompt=negative_prompt,
        height=height,
        width=width,
        num_inference_steps=num_inference_steps,
        guidance_scale=prior_guidance_scale,
        num_images_per_prompt=num_images_per_prompt,  # Assuming this is supported
        generator=generator
    )

    # Extract image embeddings from the generator object
    image_embeddings = None
    for result in prior_results:
        if 'image_embeddings' in result:
            image_embeddings = result['image_embeddings']
            break

    if image_embeddings is None:
        raise ValueError("Failed to generate image embeddings from the prior pipeline")

    # Generate with Decoder
    decoder_output = decoder_pipeline(
        image_embeddings=image_embeddings,
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=decoder_guidance_scale,
        num_images_per_prompt=num_images_per_prompt,  # Assuming this is correctly applied
        generator=generator,
        output_type="pil"
    )

    return decoder_output.images[0]  # Returns the first (and expected only) generated image

# Example usage
if __name__ == "__main__":
    prompt = pos_prompt
    negative_prompt = neg_prompt
    Random_seed = random.randint(0, 6969420)  # Ensure you have imported random and generated Random_seed

    # Truncate the prompt to a reasonable length and replace spaces with underscores
    safe_prompt = re.sub(r'[^\w\s-]', '', prompt)[:50].strip().replace(' ', '_')

    # Generate the image
    image = generate_image(prompt, negative_prompt, num_images_per_prompt=Unum_images_per_prompt, seed=Random_seed, width=Cwidth, height=Cheight, prior_guidance_scale=Uprior_guidance_scale, decoder_guidance_scale=Udecoder_guidance_scale, num_inference_steps=Unum_inference_steps)

    # Format the filename with the sanitized prompt and seed
    filename = f"/content/{safe_prompt}_{Random_seed}.png"

    # Save the image
    image.save(filename)
    print(f"Image saved as {filename}")
    image.show()

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Image saved as /content/2RAW_photo_of_a_musclebound_chimpanzee_wearing_hip_1037539.png


# Save all Images to google drive (do this before you quit)

In [64]:
import os
import shutil

def copy_files(source_path, dest_path):
    # Create the destination directory if it doesn't exist
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)

    # Process each file from the source directory
    for filename in os.listdir(source_path):
        file_path = os.path.join(source_path, filename)
        # Check if the entity is a file and has a .png extension
        if os.path.isfile(file_path) and filename.lower().endswith('.png'):
            shutil.copy(file_path, dest_path)
            print(f'Copied {filename}')

# Source and destination paths
source_dir = '/content'
dest_dir = '/content/drive/MyDrive/cascade_output4'

# Copy all .png files from source to destination
copy_files(source_dir, dest_dir)

print('PNG files copied successfully.')

Copied 2RAW_photo_of_a_cool_chimp_wearing_baggy_jeans_and_3496697.png
Copied 2RAW_photo_of_a_musclebound_chimpanzee_gangster_mo_256886.png
Copied 2RAW_photo_of_a_chimp_gang_boss_wearing_thug_cloth_2313325.png
Copied 2RAW_photo_of_a_chimp_gang_boss_wearing_thug_cloth_4108133.png
Copied 2RAW_photo_of_a_cool_chimp_wearing_baggy_jeans_and_2038295.png
Copied 2RAW_photo_of_a_musclebound_chimpanzee_gangster_mo_6042952.png
Copied 2RAW_photo_of_a_cool_chimp_wearing_hawaiian_shorts_6942408.png
Copied 2RAW_photo_of_a_cool_chimp_wearing_hawaiian_shorts_4057706.png
Copied 2RAW_photo_of_a_musclebound_chimpanzee_gangster_mo_2200027.png
Copied 2RAW_photo_of_a_cool_chimp_wearing_pinstripe_outfi_2237926.png
Copied 2RAW_photo_of_a_cool_chimp_wearing_hawaiian_shorts_3287287.png
Copied 2RAW_photo_of_a_chimp_gang_boss_wearing_thug_cloth_1614776.png
Copied 2RAW_photo_of_a_gang_chimp_wearing_street_clothing_1252962.png
Copied 2RAW_photo_of_a_cool_chimp_wearing_pinstripe_outfi_3921726.png
Copied 2RAW_photo_of_